In [ ]:
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/master
!pip3 install --upgrade pip

In [ ]:
!pip3 install waymo-open-dataset
import os
import tensorflow as tf
import math
import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import gridspec
import sys
import gc
import csv
import glob
from os import path


#tf.enable_eager_execution()

# from waymo_open_dataset.utils import range_image_utils
# from waymo_open_dataset.utils import transform_utils
# from waymo_open_dataset.utils import  frame_utils
# from waymo_open_dataset import dataset_pb2 as open_dataset

In [ ]:
# Connect to Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#dependencies
!git clone https://github.com/gdlg/simple-waymo-open-dataset-reader.git dataset-reader
%cd /content/dataset-reader/
!python3 setup.py install

In [ ]:
from simple_waymo_open_dataset_reader import WaymoDataFileReader
from simple_waymo_open_dataset_reader import dataset_pb2, label_pb2
from simple_waymo_open_dataset_reader import utils
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler


#FEATURE EXTRACTION FROM Modified simple-waymo-open-dataset-reader


version = "[v4-0001]-"

# Grab folder list
%cd "/content/gdrive/My Drive/Waymo/training/"
folders = glob.glob("*")
if "desktop.ini" in folders:
  folders.remove("desktop.ini")
folders.sort()

fold_count = 1

# Iterate through folders
for folder in folders:
  %cd {folder}
  # Returns a datetime object containing the local date and time
  dateTimeObj = datetime.now()
  
  print('|--- + -{ ' + str(dateTimeObj) + " }- " + str(folder) + " -{ " + str(fold_count) + " of " + str(len(folders)) + " }-")
  fold_count+=1

  # Grab .tfrecord files in folder
  dataset_files               = glob.glob("*.tfrecord")
  dataset_files.sort()
  
  file_count = 1

  if (path.exists(str(version) + str(folder)+(str(".csv")))):
    %cd "/content/gdrive/My Drive/Waymo/training/" 
    continue
  
  # Open a new csv file to write (Same name as the folder)
  with open(str(version) + str(folder)+(str(".csv")), "w", newline='\n') as write_file:
    for data_set in dataset_files:
      # Begin generating required features from each dataset file
      dateTimeObj = datetime.now()
      print('|    | --- + -{ ' + str(dateTimeObj) + " }- " + str(data_set) + " -{ " + str(file_count) + " of " + str(len(dataset_files)) + " }-")
      file_count+=1

      f                       = csv.writer(write_file)
      
      datafile                = WaymoDataFileReader(data_set)

      # Generate a table of the offset of all frame records in the file.
      table                   = datafile.get_record_table()

      # Dictionary mapping each object to a unique tracking ID.
      object_ids              = dict()

      # velocity lag variables
      lag_vx, lag_vy, lag_vz, lag_ts  = 0 , 0 , 0 , 0
      
      # Loop through the whole file
      ## and dump the label information.
      
      # %mkdir {data_set[:-9]}
      # %cd {data_set[:-9]}
      
      # accel variables
      acc_x = [] # acc in x direction
      acc_y = []
      acc_z = []
      acc = [] # overall accel
      dt_all = []
      lag_accx, lag_accy, lag_accz  = 0 , 0 , 0
      
      lag_frame = 0
      
      for frameno,frame in enumerate(datafile):
        # Obtain timestamp
        timestamp             = frame.images[0].pose_timestamp
        
        # Velocity and Angular Velocity Values
        v_x , v_y, v_z        = frame.images[0].velocity.v_x, frame.images[0].velocity.v_y, frame.images[0].velocity.v_z
        w_x , w_y, w_z        = frame.images[0].velocity.w_x, frame.images[0].velocity.w_y, frame.images[0].velocity.w_z


        if frameno != lag_frame:  
          lag_vx, lag_vy, lag_vz , lag_ts = backup_vx, backup_vy, backup_vz, backup_ts
          lag_accx, lag_accy, lag_accz = backup_accx, backup_accy, backup_accz
        
        if frameno == 0:
          acc_x = 0
          acc_y = 0
          acc_z = 0
          acc = 0
          dt = 0
          lag_vx, lag_vy, lag_vz , lag_ts = 0 , 0 , 0, 0
        else:
          dt = timestamp - lag_ts # time diff between consecutive frames
          dv_x = v_x - lag_vx   # dv in x direction
          dv_y = v_y - lag_vy
          dv_z = v_z - lag_vz
          dv = (dv_x**2) + (dv_y**2) + (dv_z**2)  # total velocity from x,y and z direction
          acc_x = (dv_x/dt)   # accel in x direction
          acc_y = (dv_y/dt)
          acc_z = (dv_z/dt)
          acc = (math.sqrt(dv)/dt) # overall acceleration
        
        # Get the top laser information
        laser_name            = dataset_pb2.LaserName.TOP
        laser                 = utils.get(frame.lasers, laser_name)
        laser_calibration     = utils.get(frame.context.laser_calibrations, laser_name)

        # Parse the top laser range image and get the associated projection.
        ri, camera_projection, range_image_pose = utils.parse_range_image_and_camera_projection(laser)

        # Convert the range image to a point cloud.
        pcl, pcl_attr         = utils.project_to_pointcloud(frame, ri, camera_projection, range_image_pose, laser_calibration)










        # camera_select = list of camera's to include 
        camera_select = [dataset_pb2.CameraName.FRONT,dataset_pb2.CameraName.FRONT_LEFT,dataset_pb2.CameraName.FRONT_RIGHT]

        object_output_front       = []
        object_output_front_left  = [] 
        object_output_front_right = [] 


        for camera_name in camera_select:
          camera_calibration    = utils.get(frame.context.camera_calibrations, camera_name)
          camera                = utils.get(frame.images, camera_name)

          # Get the transformation matrix for the camera.
          vehicle_to_image      = utils.get_image_transform(camera_calibration)

          # Decode the image
          # img                   = utils.decode_image(camera)

          jpg_name              = str(data_set) + '--' + str(frameno) + '.jpg'
          
          # with tf.compat.v1.Session() as sess:
          #   decode_img          = tf.io.decode_jpeg(frame.images[0].image, channels=3)
          #   new_img             = tf.image.resize(decode_img, [224,224], method='nearest')
          #   enc                 = tf.io.encode_jpeg(new_img)
          #   const_name          = tf.constant(jpg_name)
          #   fwrite              = tf.io.write_file(const_name, enc)
          #   result              = sess.run(fwrite)

          # Some of the labels might be fully hidden therefore we attempt to compute the label visibility
          # by counting the number of LIDAR points inside each label bounding box.

          # For each label, compute the transformation matrix from the vehicle space to the box space.
          vehicle_to_labels     = [np.linalg.inv(utils.get_box_transformation_matrix(label.box)) for label in frame.laser_labels]
          vehicle_to_labels     = np.stack(vehicle_to_labels)

          # Convert the pointcloud to homogeneous coordinates.
          pcl1                  = np.concatenate((pcl,np.ones_like(pcl[:,0:1])),axis=1)

          # Transform the point cloud to the label space for each label.
          # proj_pcl shape is [label, LIDAR point, coordinates]
          proj_pcl              = np.einsum('lij,bj->lbi', vehicle_to_labels, pcl1)

          # For each pair of LIDAR point & label, check if the point is inside the label's box.
          # mask shape is [label, LIDAR point]
          mask                  = np.logical_and.reduce(np.logical_and(proj_pcl >= -1, proj_pcl <= 1),axis=2)

          # Count the points inside each label's box
          counts                = mask.sum(1)

          # Keep boxes which contain at least 10 LIDAR points.
          visibility            = counts > 10

          # Get the transformation matrix from vehicle space to camera space.
          camera_extrinsic      = np.array(camera_calibration.extrinsic.transform).reshape(4,4)
          camera_extrinsic_inv  = np.linalg.inv(camera_extrinsic)

          # For each non occluded label
          for label,labelvis in zip(frame.laser_labels,visibility):
            if not labelvis:
                continue

            # Assign a category name to the label
            object_type = None
            if label.type == label_pb2.Label.Type.TYPE_VEHICLE:
                object_type = 2
            elif label.type == label_pb2.Label.Type.TYPE_PEDESTRIAN:
                object_type = 0
            elif label.type == label_pb2.Label.Type.TYPE_SIGN:
                object_type = 3
            elif label.type == label_pb2.Label.Type.TYPE_CYCLIST:
                object_type = 1
            else:
                continue

            # If this is a new object, create a tracking ID for it.
            if label.id not in object_ids:
                object_ids[label.id] = len(object_ids)

            # Transform the label position to the camera space.
            pos                 = (label.box.center_x,label.box.center_y,label.box.center_z,1)
            pos                 = np.matmul(camera_extrinsic_inv, pos)

            # Compute the relative angle

            # relative_angle      = label.box.heading - math.atan2(pos[2],pos[0])

            # Calculate Distance

            dist_to_object = math.sqrt( pos[0]**2 + pos[1]**2 + pos[2]**2 )
            
            # Depending on camera - insert into correct list
            if camera_name == 1:
              object_output_front.append([object_type, dist_to_object])
            elif camera_name == 2:
              object_output_front_left.append([object_type, dist_to_object])
            elif camera_name == 3:
              object_output_front_right.append([object_type, dist_to_object])
                                  
        # PROCESS LISTS OF OBJECTS HERE
        types_of_objects = [0,1,2,3]

        dist_for_objs_front = [[],[],[],[]]

        for object_type in types_of_objects:
          for row in object_output_front:
            if object_type == 0 and row[0] == 0: # TYPE_PEDESTRIAN
              dist_for_objs_front[0].append(row[1])
            elif object_type == 1 and row[0] == 1: # TYPE_CYCLIST
              dist_for_objs_front[1].append(row[1])
            elif object_type == 2 and row[0] == 2: # TYPE_VEHICLE
              dist_for_objs_front[2].append(row[1])
            elif object_type == 3 and row[0] == 3: # TYPE_SIGN
              dist_for_objs_front[3].append(row[1])

        final_output_front = []

        for row in dist_for_objs_front:
          if len(row) == 0:
            final_output_front.append(-1)
          else:
            scaler = MinMaxScaler()
            row = np.array(row)
            row = row.reshape(-1, 1)
            scaler.fit(row)
            temp = scaler.transform(row)
            dist = np.mean(temp)
            final_output_front.append(dist)
        
        # print("final_output_front: " + str(final_output_front))
        
        dist_for_objs_front_left = [[],[],[],[]]

        for object_type in types_of_objects:
          for row in object_output_front_left:
            if object_type == 0 and row[0] == 0: # TYPE_PEDESTRIAN
              dist_for_objs_front_left[0].append(row[1])
            elif object_type == 1 and row[0] == 1: # TYPE_CYCLIST
              dist_for_objs_front_left[1].append(row[1])
            elif object_type == 2 and row[0] == 2: # TYPE_VEHICLE
              dist_for_objs_front_left[2].append(row[1])
            elif object_type == 3 and row[0] == 3: # TYPE_SIGN
              dist_for_objs_front_left[3].append(row[1])

        final_output_front_left = []

        for row in dist_for_objs_front_left:
          if len(row) == 0:
            final_output_front_left.append(-1)
          else:
            scaler = MinMaxScaler()
            row = np.array(row)
            row = row.reshape(-1, 1)
            scaler.fit(row)
            temp = scaler.transform(row)
            dist = np.mean(temp)
            final_output_front_left.append(dist)
        
        # print("final_output_front_left: " + str(final_output_front_left))


        dist_for_objs_front_right = [[],[],[],[]]

        for object_type in types_of_objects:
          for row in object_output_front_right:
            if object_type == 0 and row[0] == 0: # TYPE_PEDESTRIAN
              dist_for_objs_front_right[0].append(row[1])
            elif object_type == 1 and row[0] == 1: # TYPE_CYCLIST
              dist_for_objs_front_right[1].append(row[1])
            elif object_type == 2 and row[0] == 2: # TYPE_VEHICLE
              dist_for_objs_front_right[2].append(row[1])
            elif object_type == 3 and row[0] == 3: # TYPE_SIGN
              dist_for_objs_front_right[3].append(row[1])
        
        final_output_front_right = []

        for row in dist_for_objs_front_right:
          if len(row) == 0:
            final_output_front_right.append(-1)
          else:
            scaler = MinMaxScaler()
            row = np.array(row)
            row = row.reshape(-1, 1)
            scaler.fit(row)
            temp = scaler.transform(row)
            dist = np.mean(temp)
            final_output_front_right.append(dist)
        
        # print("final_output_front_right: " + str(final_output_front_right))

        head = [frameno,timestamp]

        veh_output          = [acc_x, acc_y, acc_z, v_x , v_y, v_z, w_x , w_y, w_z,
                              lag_vx, lag_vy, lag_vz, lag_accx, lag_accy, lag_accz, 
                              "/Waymo/training/" + str(folder) + "/" + str(data_set) + "/" + str(jpg_name)]
        
        output = head + final_output_front + final_output_front_left + final_output_front_right + veh_output
        
        f.writerow(output)

        # Update Lag Variables
        if frameno != 0:
          lag_vx, lag_vy, lag_vz , lag_ts = v_x, v_y, v_z, timestamp
          lag_accx, lag_accy, lag_accz = acc_x, acc_y, acc_z

        backup_vx, backup_vy, backup_vz, backup_ts = v_x, v_y, v_z, timestamp
        backup_accx, backup_accy, backup_accz = acc_x, acc_y, acc_z
        # the camera loop ends here.
      %cd "/content/gdrive/My Drive/Waymo/training/{folder}/"
    write_file.close()
  %cd "/content/gdrive/My Drive/Waymo/training/"

/content/gdrive/My Drive/Waymo/training
/content/gdrive/My Drive/Waymo/training/training_0000
|--- + -{ 2020-04-18 03:39:14.881868 }- training_0000 -{ 1 of 32 }-
/content/gdrive/My Drive/Waymo/training
/content/gdrive/My Drive/Waymo/training/training_0001
|--- + -{ 2020-04-18 03:39:15.489345 }- training_0001 -{ 2 of 32 }-
|    | --- + -{ 2020-04-18 03:39:16.018257 }- segment-10485926982439064520_4980_000_5000_000_with_camera_labels.tfrecord -{ 1 of 24 }-
/content/gdrive/My Drive/Waymo/training/training_0001
|    | --- + -{ 2020-04-18 04:38:19.410412 }- segment-10498013744573185290_1240_000_1260_000_with_camera_labels.tfrecord -{ 2 of 24 }-
/content/gdrive/My Drive/Waymo/training/training_0001
|    | --- + -{ 2020-04-18 05:21:44.867558 }- segment-10500357041547037089_1474_800_1494_800_with_camera_labels.tfrecord -{ 3 of 24 }-
/content/gdrive/My Drive/Waymo/training/training_0001
|    | --- + -{ 2020-04-18 06:15:15.377459 }- segment-10517728057304349900_3360_000_3380_000_with_camera_labe